<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Neural Collaborative Filtering on Snack_Fake dataset.

Neural Collaborative Filtering (NCF) is a well known recommendation algorithm that generalizes the matrix factorization problem with multi-layer perceptron. 

This notebook provides an example of how to utilize and evaluate NCF implementation in the `reco_utils`. We use a smaller dataset in this example to run NCF efficiently with GPU acceleration on a [Data Science Virtual Machine](https://azure.microsoft.com/en-gb/services/virtual-machines/data-science-virtual-machines/).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")
import time
import pandas as pd
import tensorflow as tf

from reco_utils.recommender.ncf.ncf_singlenode import NCF
from reco_utils.recommender.ncf.dataset import Dataset as NCFDataset
from reco_utils.dataset import movielens
from reco_utils.common.notebook_utils import is_jupyter
from reco_utils.dataset.python_splitters import python_chrono_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.6.10 |Anaconda, Inc.| (default, May  8 2020, 02:54:21) 
[GCC 7.3.0]
Pandas version: 0.25.3
Tensorflow version: 1.15.2


Set the default parameters.

In [17]:
# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS = 50
BATCH_SIZE = 256

SEED = 42

### 1. Upload the fake data we generate

In [18]:
df = pd.read_csv('~/notebooks/A3_Data/snack_ncf_fake.csv')

### 2. Split the data using the Spark chronological splitter provided in utilities

In [27]:
df.head()

,userID,itemID,rating,timestamp,products
0,100,8,8,2019-11-25 22:44:14,Bokksu
1,62,8,4,2020-01-03 04:49:20,Bokksu
2,41,6,8,2020-05-12 10:40:29,Yummy Bazaar World Sampler
3,2,9,6,2020-02-14 03:14:02,MunchPak
4,49,7,8,2020-06-10 01:42:05,FitSnack


In [28]:
train, test = python_chrono_split(df, 0.75)

Generate an NCF dataset object from the data subsets.

In [29]:
data = NCFDataset(train=train, test=test, seed=SEED)

../../reco_utils/recommender/ncf/dataset.py:210: UserWarning: n_neg_test is larger than negative items set size! We will set n_neg as the smallest size: 3
  % min_num


### 3. Train the NCF model on the training data, and get the top-k recommendations for our testing data

NCF accepts implicit feedback and generates prospensity of items to be recommended to users in the scale of 0 to 1. A recommended item list can then be generated based on the scores. Note that this quickstart notebook is using a smaller number of epochs to reduce time for training. As a consequence, the model performance will be slighlty deteriorated. 

In [22]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [23]:
start_time = time.time()

model.fit(data)

train_time = time.time() - start_time

print("Took {} seconds for training.".format(train_time))

Took 1.9937241077423096 seconds for training.


In the filter out the snack user already rated

In [30]:
start_time = time.time()

users, items, preds = [], [], []
item = list(train.itemID.unique())
for user in train.userID.unique():
    user = [user] * len(item) 
    users.extend(user)
    items.extend(item)
    preds.extend(list(model.predict(user, item, is_list=True)))

all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

test_time = time.time() - start_time
print("Took {} seconds for prediction.".format(test_time))

Took 0.05442667007446289 seconds for prediction.


### 4. Evaluate how well NCF performs

In [31]:
all_predictions.head()

,userID,itemID,timestamp,products,prediction
754,1,9,NaN,NaN,0.350738
755,1,13,NaN,NaN,0.044026
756,1,7,NaN,NaN,0.045765
757,1,8,NaN,NaN,0.105393
758,1,12,NaN,NaN,0.261199


The ranking metrics are used for evaluation.

In [25]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.219986
NDCG:	0.349363
Precision@K:	0.136000
Recall@K:	0.558500


In [26]:
if is_jupyter():
    # Record results with papermill for tests
    import papermill as pm
    pm.record("map", eval_map)
    pm.record("ndcg", eval_ndcg)
    pm.record("precision", eval_precision)
    pm.record("recall", eval_recall)
    pm.record("train_time", train_time)
    pm.record("test_time", test_time)

/anaconda/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  after removing the cwd from sys.path.


/anaconda/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  """


/anaconda/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  


/anaconda/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  import sys


/anaconda/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  


/anaconda/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  if __name__ == '__main__':
